# 自定义 Embeddings

LangChain 已与许多 [第三方 embedding 模型](/docs/integrations/text_embedding/) 集成。在本指南中，我们将展示如何创建自定义 Embedding 类，以防内置类尚不存在。Embeddings 在自然语言处理应用中至关重要，因为它们将文本转换为算法可以理解的数值形式，从而支持各种应用，如相似性搜索、文本分类和聚类。

使用标准的 [Embeddings](https://python.langchain.com/api_reference/core/embeddings/langchain_core.embeddings.embeddings.Embeddings.html) 接口实现 embeddings 将允许您的 embeddings 在现有的 `LangChain` 抽象中使用（例如，作为[VectorStore](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html) 的底层 embeddings 或使用 [CacheBackedEmbeddings](/docs/how_to/caching_embeddings/) 进行缓存）。

## Interface

LangChain 中当前的 `Embeddings` 抽象旨在处理文本数据。在此实现中，输入是单个字符串或字符串列表，输出是数值数组（向量）的列表，其中每个向量代表
将输入文本嵌入到某个 n 维空间。

您的自定义 embedding 必须实现以下方法：

| 方法/属性                 | 描述                                       | 必需/可选 |
|-----------------------------|--------------------------------------------|-----------|
| `embed_documents(texts)`    | 为字符串列表生成 embeddings。              | 必需      |
| `embed_query(text)`         | 为单个文本查询生成 embedding。             | 必需      |
| `aembed_documents(texts)`   | 异步为字符串列表生成 embeddings。         | 可选      |
| `aembed_query(text)`        | 异步为单个文本查询生成 embedding。        | 可选      |

这些方法确保您的 embedding 模型可以无缝集成到 LangChain 框架中，提供同步和异步功能以优化可扩展性和性能。

:::note
`Embeddings` 目前不实现 [Runnable](/docs/concepts/runnables/) 接口，也不是 pydantic `BaseModel` 的实例。
:::

### Embedding 查询与文档

`embed_query` 和 `embed_documents` 方法是必需的。这些方法都处理
字符串输入。出于历史原因，vector store 使用 embedding 模型来访问 `Document.page_content` 属性。

`embed_query` 接受单个字符串并返回一个单一的 embedding，表示为浮点数列表。
如果您的模型在嵌入查询与底层文档方面有不同的模式，您可以
实现此方法来处理这种情况。

`embed_documents` 接受字符串列表并返回 embedding 列表，表示为浮点数列表的列表。

:::note
`embed_documents` 接受纯文本列表，而不是 LangChain `Document` 对象列表。此方法的名称
可能在 LangChain 的未来版本中更改。
:::

## 实现

例如，我们将实现一个返回常量向量的简单嵌入模型。此模型仅用于说明目的。

In [1]:
from typing import List

from langchain_core.embeddings import Embeddings


class ParrotLinkEmbeddings(Embeddings):
    """ParrotLink embedding model integration.

    # TODO: Populate with relevant params.
    Key init args — completion params:
        model: str
            Name of ParrotLink model to use.

    See full list of supported init args and their descriptions in the params section.

    # TODO: Replace with relevant init params.
    Instantiate:
        .. code-block:: python

            from langchain_parrot_link import ParrotLinkEmbeddings

            embed = ParrotLinkEmbeddings(
                model="...",
                # api_key="...",
                # other params...
            )

    Embed single text:
        .. code-block:: python

            input_text = "The meaning of life is 42"
            embed.embed_query(input_text)

        .. code-block:: python

            # TODO: Example output.

    # TODO: Delete if token-level streaming isn't supported.
    Embed multiple text:
        .. code-block:: python

             input_texts = ["Document 1...", "Document 2..."]
            embed.embed_documents(input_texts)

        .. code-block:: python

            # TODO: Example output.

    # TODO: Delete if native async isn't supported.
    Async:
        .. code-block:: python

            await embed.aembed_query(input_text)

            # multiple:
            # await embed.aembed_documents(input_texts)

        .. code-block:: python

            # TODO: Example output.

    """

    def __init__(self, model: str):
        self.model = model

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed search docs."""
        return [[0.5, 0.6, 0.7] for _ in texts]

    def embed_query(self, text: str) -> List[float]:
        """Embed query text."""
        return self.embed_documents([text])[0]

    # optional: add custom async implementations here
    # you can also delete these, and the base class will
    # use the default implementation, which calls the sync
    # version in an async executor:

    # async def aembed_documents(self, texts: List[str]) -> List[List[float]]:
    #     """Asynchronous Embed search docs."""
    #     ...

    # async def aembed_query(self, text: str) -> List[float]:
    #     """Asynchronous Embed query text."""
    #     ...

### 让我们来测试一下 

🧪

In [2]:
embeddings = ParrotLinkEmbeddings("test-model")
print(embeddings.embed_documents(["Hello", "world"]))
print(embeddings.embed_query("Hello"))

[[0.5, 0.6, 0.7], [0.5, 0.6, 0.7]]
[0.5, 0.6, 0.7]


## 贡献

我们欢迎将 Embedding 模型贡献给 LangChain 代码库。

如果您旨在为新的提供商（例如，具有新依赖项或 SDK 的提供商）贡献 Embedding 模型，我们鼓励您在单独的 `langchain-*` 集成包中发布您的实现。这将使您能够妥善管理依赖项并对您的包进行版本控制。请参阅我们的[贡献指南](/docs/contributing/how_to/integrations/)了解此过程的完整说明。